In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
##FixPath
os.chdir(""")

In [3]:
##Read Data
product_ = pd.read_csv("Products.csv")
relevency_table = pd.read_csv("Relevency_table.csv")
exclusion_ = pd.read_csv("Exclusion.csv")

### CheckData

In [4]:
product_.head()

,product,volume
0,5650512,150
1,5650506,600
2,5649630,300
3,5650672,200
4,5650537,100


In [5]:
relevency_table.head()

,customers,product,relevancy_score
0,A10001,5649565,0.293978
1,A10001,5649585,0.076184
2,A10001,5649607,0.312285
3,A10001,5649625,0.113652
4,A10001,5649630,0.108481


In [6]:
x = relevency_table.groupby('customers')['customers'].count()

In [7]:
product_count = x.reset_index(name='values')

In [8]:
product_count.query('values < 10')##Count looks ok

,customers,values
563,A10564,8
622,A10623,8
768,A10769,9
775,A10776,9
784,A10785,9


In [190]:
relevency_table.head()

,customers,product,relevancy_score
0,A10001,5649565,0.293978
1,A10001,5649585,0.076184
2,A10001,5649607,0.312285
3,A10001,5649625,0.113652
4,A10001,5649630,0.108481


In [191]:
exclusion_.head()

,product1,product2
0,5649565,5649646
1,5649585,5649910
2,5649585,5649921
3,5649607,5649931
4,5649607,5649929


### Making Sperate matrix to put under CVXPY

In [11]:
product_list = product_['product'].unique().tolist()

In [12]:
P = np.ones((len(product_list), len(product_list)))

In [13]:
###Product Matrix
for i in range(0,len(product_list)):
    prod = product_list[i]
    base = exclusion_[exclusion_['product1'] == prod]
    tolist = base['product2'].tolist()
    P[i][i]=0
    for shit in tolist:
        j = product_list.index(shit)
        P[i][j]=0

In [14]:
customer_list = relevency_table['customers'].unique().tolist()

In [42]:
###Relivancy Materix
R = np.zeros((len(customer_list), len(product_list)))
#C_P = np.zeros((len(customer_list), len(product_list)))

In [43]:
for i in range(0,len(customer_list)):
    cust = customer_list[i]
    base = relevency_table[relevency_table['customers'] == cust]
    relevancy_prd = base['product'].tolist()
    relevancy_scr = base['relevancy_score'].tolist()
    for j in range(0,len(relevancy_prd)):
        prod_index = product_list.index(relevancy_prd[j])
        R[i][prod_index]=relevancy_scr[j]

In [44]:
summat = np.ones((len(customer_list),1))

 ### Goal is to maximize relevancy_score*Product(Match.Unmatch)*Boolean Flag, Summat taken to take the whole sum

In [45]:
import cvxpy as cp

In [154]:
solve_for = cp.Variable((len(product_list),len(customer_list)),boolean=True)

In [155]:
objective = cp.Maximize(cp.sum((R@P@solve_for)@summat))

#### 1st criteria availability of product

In [156]:
supply_constr = []
for i in range(0,len(product_list)):
    prod_value = product_[product_['product']==product_list[i]]
    supply_constr += [cp.sum(solve_for[i,])<=(prod_value['volume'][i])]

#### 2nd criteria Max Prod is 8, Min is 3

In [157]:
#cart_constr = []
for i in range(0,len(customer_list)):
    supply_constr +=[cp.sum(solve_for[:,i])<=8,cp.sum(solve_for[:,i])>=3]

#### Who likes what, i.e what is relevant

In [192]:
for i in range(0,len(customer_list)):
    cust = customer_list[i]
    base = relevency_table[relevency_table['customers'] == cust]
    prod_pos = base['product'].tolist()
    indexing = np.where(np.isin(prod_pos,product_list))[0].tolist()
    for j in range(0,len(product_list)):
        if (j not in indexing):
            supply_constr += [solve_for[j,i]==0]

In [193]:
constraints = supply_constr

In [195]:
prob = cp.Problem(objective, constraints)

In [161]:
np.random.seed(1)

In [162]:
### Since this is MLIP (We need help GLKP)
import cvxopt

In [163]:
result = prob.solve()

In [165]:
result

25251924.05531975

In [166]:
x = solve_for.value

In [167]:
np.sum(x,axis = 0)

array([8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8.

In [168]:
np.sum(x,axis = 1)

array([150.,  10.,   5.,   5., 100.,  60.,   2.,   7., 200., 200.,   1.,
        50., 100.,   0., 143., 200., 200.,   0.,   0.,   0.,   0.,   0.,
         0., 100.,   0.,   0., 100., 100.,   0., 200.,   0., 300.,   0.,
         0.,   0.,  50.,   0.,   0., 400.,   0.,   0., 200., 150.,   0.,
         0.,   0., 300.,   0., 100.,   0.,   0.,   0.,   0.,   0., 250.,
         0.,   0.,   0.,   0., 200., 100., 200.,   0., 300.,   0.,   0.,
       100., 150.,   0., 200.,   0.,   0.,   0.,   0., 200., 244.,   0.,
       100.,   0., 193.,   0.,   0.,   0.,   0., 150.,   0., 127., 117.,
        50.,   0.,  75.,   0.,   0.,  55.,   0.,   0.,   0.,  30.,   0.,
         0.,  14.,   0.,   8.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.])

In [169]:
np.sum(R)

4034.2341626549996

In [170]:
customer_back = []
relevance_back = []
prod_back = []
for i in range(0,len(customer_list)):
    cust = customer_list[i]
    customer_back.append([cust]*len(product_list))
    relevance_back.append(x[:,i])
    prod_back.append(product_list)  

In [171]:
mydf = pd.DataFrame(list(zip(np.array(customer_back).flat, np.array(relevance_back).flat,np.array(prod_back).flat)), columns = ['customers', 'INCART','product'])

In [172]:
mydf.head()

,customers,INCART,product
0,A10001,0.0,5650512
1,A10001,0.0,5650506
2,A10001,0.0,5649630
3,A10001,0.0,5650672
4,A10001,0.0,5650537


In [173]:
submission = pd.merge(relevency_table,mydf,on=['customers','product'],how='left')

In [174]:
submission = submission[submission['INCART']==1]

In [175]:
submission['relevancy_score'].sum()

176.378050743

In [176]:
x = submission.groupby('customers')['customers'].count().reset_index(name='values')

In [180]:
x = x.sort_values('values')

In [181]:
submission_2 = pd.merge(submission,x,on = 'customers',how = 'left') 

In [184]:
submission_2 = submission_2[submission_2['values']>2]

In [185]:
submission_2 = submission_2.drop(['INCART', 'values'], axis=1)

In [189]:
submission_2.to_csv("submission.csv",index=False)